In [1]:
STUDENT_ID = "Александр Сапожников 411"

In [2]:
# подготовка данных
import pandas as pd
import pathlib
import hashlib
import numpy as np
import random


train_directory = pathlib.Path("data/train")
sample_size = 5000

def initialize_random_seed():
    """Инициализирует ГПСЧ из STUDENT_ID"""
    sha256 = hashlib.sha256()
    sha256.update(STUDENT_ID.encode("utf-8"))
    
    fingerprint = int(sha256.hexdigest(), 16) % (2**32) 
    
    random.seed(fingerprint)
    np.random.seed(fingerprint)


def read_target_variable():
    """Прочитаем разметку фотографий из названий файлов"""
    target_variable = {
        "filename": [],
        "is_cat": []
    }
    image_paths = list(train_directory.glob("*.jpg"))
    random.shuffle(image_paths)
    for image_path in image_paths[:sample_size]:
        filename = image_path.name
        class_name = filename.split(".")[0]
        target_variable["filename"].append(filename)
        target_variable["is_cat"].append(class_name == "cat")

    return pd.DataFrame(data=target_variable)


initialize_random_seed()

target_df = read_target_variable()
target_df


,filename,is_cat
0,dog.9541.jpg,False
1,dog.3423.jpg,False
2,cat.2157.jpg,True
3,dog.11885.jpg,False
4,cat.10599.jpg,True
...,...,...
4995,dog.6624.jpg,False
4996,dog.5719.jpg,False
4997,dog.2286.jpg,False
4998,dog.12266.jpg,False


In [3]:
# подготовка данных
from PIL import Image
from tqdm import tqdm_notebook


def read_data(target_df):
    """Читает данные изображений и строит их признаковое описание"""
    image_size = (100, 100)
    features = []
    target = []
    for i, image_name, is_cat in tqdm_notebook(target_df.itertuples(), total=len(target_df)):
        image_path = str(train_directory / image_name)
        image = Image.open(image_path)
        image = image.resize(image_size) # уменьшаем изображения
        image = image.convert('LA') # преобразуем в Ч\Б
        pixels = np.asarray(image)[:, :, 0]
        pixels = pixels.flatten()
        features.append(pixels)
        target.append(is_cat)
    return np.array(features), np.array(target)

features, target = read_data(target_df)
features

<ipython-input-3-67ebf5be4e61>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, image_name, is_cat in tqdm_notebook(target_df.itertuples(), total=len(target_df)):


  0%|          | 0/5000 [00:00<?, ?it/s]

array([[ 93, 103,  92, ...,  63,  49,  29],
       [ 87,  89,  89, ..., 118, 118, 122],
       [ 86,  85,  85, ...,  43,  54,  69],
       ...,
       [ 86,  76,  80, ...,  25,  26,  22],
       [166, 168, 169, ..., 228, 225, 210],
       [168, 166, 165, ..., 254, 254, 255]], dtype=uint8)

In [4]:
# разбиваем выборку на test, train, valid

from sklearn.model_selection import train_test_split

featuresTrain, featuresTest, targetTrain, targetTest =  train_test_split(features, target, test_size = 0.4)

featuresTest, featuresValid, targetTest, targetValid =  train_test_split(featuresTest, targetTest, test_size = 0.5)

In [13]:
# обучаем SGD и изменяем параметр eta

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

n = 100

# у нас будет n сетей, результат каждой сохраним
exact_mas = []
train_mas = []

int_targetTest = [int(elem) for elem in targetTest]
int_targetTrain = [int(elem) for elem in targetTrain]
for i in range(n): 
    eta = (i+1) / n
    print(eta)
    clf = make_pipeline(StandardScaler(), SGDClassifier(learning_rate="constant", eta0 = eta))
    clf.fit(featuresTrain, targetTrain)
    
    predict = clf.predict(featuresTest)
    result_learn = accuracy_score(int_targetTest, [int(elem) for elem in predict])
    print(result_learn)
    exact_mas.append(result_learn)
    
    predict = clf.predict(featuresTrain)
    result_learn = accuracy_score(int_targetTrain, [int(elem) for elem in predict])
    print(result_learn)
    train_mas.append(result_learn)

0.01
0.517
0.905
0.02
0.497
0.832
0.03
0.498
0.8633333333333333
0.04
0.5
0.8493333333333334
0.05
0.508
0.8616666666666667
0.06
0.508
0.816
0.07
0.507
0.8023333333333333
0.08
0.51
0.823
0.09
0.504
0.7516666666666667
0.1
0.498
0.8156666666666667
0.11
0.508
0.798
0.12
0.491
0.7273333333333334
0.13
0.509
0.811
0.14
0.506
0.7706666666666667
0.15
0.501
0.7813333333333333
0.16
0.495
0.7286666666666667
0.17
0.514
0.7523333333333333
0.18
0.482
0.6773333333333333
0.19
0.507
0.7046666666666667
0.2
0.517
0.731
0.21
0.478
0.7473333333333333
0.22
0.499
0.7293333333333333
0.23
0.507
0.7163333333333334
0.24
0.486
0.72
0.25
0.509
0.7256666666666667
0.26
0.498
0.716
0.27
0.503
0.7556666666666667
0.28
0.514
0.717
0.29
0.544
0.74
0.3
0.473
0.6553333333333333
0.31
0.507
0.735
0.32
0.506
0.7256666666666667
0.33
0.507
0.6426666666666667
0.34
0.497
0.6763333333333333
0.35
0.519
0.6986666666666667
0.36
0.511
0.669
0.37
0.522
0.6643333333333333
0.38
0.487
0.6593333333333333
0.39
0.489
0.6913333333333334
0.4
0.5

In [14]:
# смотрим какой параметр eta оказался лучше

max_accuracy = np.max(exact_mas)
index_max_accuracy = exact_mas.index(max_accuracy)
print(max_accuracy, index_max_accuracy)

0.553 73


In [17]:
# нормальзуем данные и пытаемся на этих данных обучить сеть с оптимальным eta

from sklearn.preprocessing import StandardScaler

normalizeFeaturesTrain = scaler.fit_transform(featuresTrain)
clf = make_pipeline(StandardScaler(), SGDClassifier(learning_rate="constant", eta0 = index_max_accuracy / n))
clf.fit(featuresTrain, targetTrain)
predict = clf.predict(normalizeFeaturesTrain)

print(accuracy_score([int(elem) for elem in targetTrain], [int(elem) for elem in predict]))

0.5006666666666667


In [20]:
# для всех eta с прошлого раза обучаем и проверяем на нормальзованных данных 

from sklearn.preprocessing import StandardScaler
n = 100
exact_mas = []
train_mas = []
scaler = StandardScaler()
normalizeFeaturesTrain = scaler.fit_transform(featuresTrain)
normalizeFeaturesTest = scaler.fit_transform(featuresTest)

int_targetTest = [int(elem) for elem in targetTest]
int_targetTrain = [int(elem) for elem in targetTrain]
normalizeFeaturesTrain = scaler.fit_transform(featuresTrain)
for i in range(n): 
    eta = (i+1) / n
    print(eta)
    clf = make_pipeline(StandardScaler(), SGDClassifier(learning_rate="constant", eta0 = eta))
    clf.fit(featuresTrain, targetTrain)
    
    predict = clf.predict(normalizeFeaturesTest)
    result_learn = accuracy_score(int_targetTest, [int(elem) for elem in predict])
    print(result_learn)
    exact_mas.append(result_learn)
    
    predict = clf.predict(normalizeFeaturesTrain)
    result_learn = accuracy_score(int_targetTrain, [int(elem) for elem in predict])
    print(result_learn)
    train_mas.append(result_learn)

# результат перобучение стала меньше и отрыв между train и test данными уменьшился

0.01
0.504
0.49933333333333335
0.02
0.496
0.5006666666666667
0.03
0.504
0.49933333333333335
0.04
0.504
0.49933333333333335
0.05
0.504
0.49933333333333335
0.06
0.504
0.49933333333333335
0.07
0.496
0.5006666666666667
0.08
0.496
0.5006666666666667
0.09
0.504
0.49933333333333335
0.1
0.496
0.5006666666666667
0.11
0.496
0.5006666666666667
0.12
0.504
0.49933333333333335
0.13
0.504
0.49933333333333335
0.14
0.496
0.5006666666666667
0.15
0.496
0.5006666666666667
0.16
0.496
0.5006666666666667
0.17
0.504
0.49933333333333335
0.18
0.496
0.5006666666666667
0.19
0.496
0.5006666666666667
0.2
0.504
0.49933333333333335
0.21
0.496
0.5006666666666667
0.22
0.504
0.49933333333333335
0.23
0.504
0.49933333333333335
0.24
0.504
0.49933333333333335
0.25
0.496
0.5006666666666667
0.26
0.504
0.49933333333333335
0.27
0.496
0.5006666666666667
0.28
0.496
0.5006666666666667
0.29
0.496
0.5006666666666667
0.3
0.496
0.5006666666666667
0.31
0.504
0.49933333333333335
0.32
0.504
0.49933333333333335
0.33
0.504
0.49933333333333

In [22]:
# перебираем параметр n_estimators для Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


rfc = RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100],
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv = 5)
CV_rfc.fit(featuresTrain, targetTrain)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]})

In [33]:
# на каком значение n_estimators Random Forest показал себя лучше

CV_rfc.best_params_

{'n_estimators': 60}

In [43]:
# используя самый лучший параметр n_estimators оцениваем качество на 5 фолдах

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import accuracy_score

rfc = RandomForestClassifier(random_state=42, n_estimators = CV_rfc.best_params_['n_estimators'])

skfolds = StratifiedKFold(n_splits=5, random_state=42, shuffle = True)

result = []

for train_index, test_index in skfolds.split(featuresTrain, targetTrain):
    clone_rfc = clone(rfc)
    X_train_folds = featuresTrain[train_index]
    y_train_folds = targetTrain[train_index]
    X_test_fold = featuresTrain[test_index]
    y_test_fold = targetTrain[test_index]
    clone_rfc.fit(X_train_folds, y_train_folds)
    
    predict = [int(elem) for elem in clone_rfc.predict(X_test_fold) ]
    ans = [int(elem) for elem in y_test_fold ]
    result.append(accuracy_score(ans, predict))

In [47]:
# среднее и стандартное отклонение точности на кросс-валидации

print(result)
result = np.array(result)
print(result.mean())
print(result.std())

[0.5983333333333334, 0.6133333333333333, 0.6133333333333333, 0.5983333333333334, 0.5683333333333334]
0.5983333333333334
0.016431676725154956
